## Importación de módulos

In [1]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO
from ultralytics.solutions import object_counter

## Aplicación interactiva

In [8]:

model = YOLO('yolov8n.pt')

video_path = "videos/traffic.mp4"
cap = cv2.VideoCapture(video_path)

track_history = defaultdict(lambda: [])
line_points = [(100, 400), (1200, 400)]  # line or region points
classes_to_count = [2,3,5,7]  # cars, motorcycles, bus, trucks

counter = object_counter.ObjectCounter()
counter.set_args(view_img=False,
                 reg_pts=line_points,
                 line_thickness=1,
                 track_thickness=1,
                 draw_tracks=True,
                 region_thickness=0,
                 count_reg_color=(255, 255, 255),
                 classes_names=model.names)

while cap.isOpened():

    success, frame = cap.read()

    if success:
        results = model.track(frame, persist=True,classes=classes_to_count,show=False)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        #annotated_frame = results[0].plot()

        # Plot the tracks and calculate speed
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y))) 
            if len(track) > 15: 
                track.pop(0)

            # Calculate speed and infer direction based on the shape of the vehicle contour and tracking direction
            if len(track) >= 2:
                current_pos = np.array(track[-1])
                previous_pos = np.array(track[-2])
                speed = np.linalg.norm(current_pos - previous_pos)

                # Infer direction based on the shape of the vehicle contour and tracking direction
                contour = np.array([[int(x), int(y)], [int(x + w), int(y)], [int(x + w), int(y + h)], [int(x), int(y + h)]])
                rect = cv2.minAreaRect(contour)
                angle = rect[2]

                # Calculate the direction of movement of the rectangle
                rect_direction = "Este" if current_pos[0] > previous_pos[0] else "Oeste"

                if -45 <= angle < 45:
                    direction = "Este" if rect_direction == "Este" else "Oeste"
                else:
                    direction = "Norte" if current_pos[1] < previous_pos[1] else "Sur"
                
                frame = counter.start_counting(frame, results)
                # Display speed and direction on the frame
                cv2.putText(frame, f"Speed: {speed:.2f} px/frame", (int(x), int(y) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                cv2.putText(frame, f"Direction: {direction}", (int(x), int(y) + 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

Line Counter Initiated.

0: 384x640 4 cars, 1 truck, 111.8ms
Speed: 3.0ms preprocess, 111.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 82.7ms
Speed: 2.5ms preprocess, 82.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 79.2ms
Speed: 2.0ms preprocess, 79.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 78.2ms
Speed: 1.0ms preprocess, 78.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 78.7ms
Speed: 2.0ms preprocess, 78.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 truck, 77.7ms
Speed: 2.0ms preprocess, 77.7ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 78.7ms
Speed: 2.5ms preprocess, 78.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 82.7ms
Speed: 